In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# تحميل ملفات البيانات
df_master = pd.read_excel("Product Matching Dataset.xlsx")
df_seller = pd.read_excel("Product Matching Dataset2.xlsx")

# تنظيف البيانات: إزالة المسافات الفارغة وتحويل النصوص إلى حروف صغيرة
df_seller["seller_item_name_clean"] = df_seller["seller_item_name"].str.strip().str.lower()
df_master["product_name_ar_clean"] = df_master["product_name_ar"].str.strip().str.lower()

# تحويل الأسماء إلى قوائم نصوص لمعالجتها
seller_names = df_seller["seller_item_name_clean"].fillna("").astype(str).tolist()
master_names = df_master["product_name_ar_clean"].fillna("").astype(str).tolist()

# إنشاء نموذج TF-IDF لمعالجة النصوص باستخدام n-grams
vectorizer = TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 4), max_features=5000)

# تحويل الأسماء إلى مصفوفة رقمية باستخدام TF-IDF
tfidf_matrix_master = vectorizer.fit_transform(master_names)

# دالة للبحث عن أقرب تطابق لاسم المنتج
def find_similar_drug(input_drug):
    input_clean = input_drug.strip().lower()  # تنظيف المدخلات
    input_vector = vectorizer.transform([input_clean])  # تحويل النص إلى تمثيل رقمي

    # حساب تشابه كوساين بين المنتج المدخل والمنتجات في القائمة
    cosine_similarities = cosine_similarity(input_vector, tfidf_matrix_master)

    # استخراج المنتج الأكثر تشابهًا
    best_match_index = cosine_similarities.argmax()
    best_score = cosine_similarities.max()

    matched_sku = df_master.iloc[best_match_index]["sku"]
    matched_name = df_master.iloc[best_match_index]["product_name_ar_clean"]

    return matched_sku, matched_name, best_score

# البحث التفاعلي عن المنتجات المشابهة
while True:
    input_drug = input("\n🔍 أدخل اسم المنتج للبحث (أو اكتب 'خروج' لإنهاء البحث): ").strip()
    if input_drug.lower() == "خروج":
        print("🚪 تم إنهاء البحث.")
        break
    
    sku, name, score = find_similar_drug(input_drug)
    print(f"\n✅ أقرب تطابق: {name} (SKU: {sku})")
    print(f"🎯 نسبة التشابه: {score * 100:.2f}%")
